In [108]:
import pickle
from pandas import read_csv

print("Loading data...")
training_file = './Data/train.p'

sign_names = read_csv("./Data/signname.csv").values[:, 1]
print(sign_names)

sign_forms = read_csv("./Data/signshape.csv").values
sign_dict = {}
for pair in sign_forms:
    sign_dict[pair[0]] = pair[1]
print(sign_dict)

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
images_train, labels_train = train['features'], train['labels']

forms_train = []
for i in range(len(labels_train)):
    forms_train.append(sign_dict.get(labels_train[i]))
    if labels_train[i] < 9:
        labels_train[i] = 0
    elif labels_train[i] >= 9:
        labels_train[i] = 1

Loading data...
['Speed limit (20km/h)' 'Speed limit (30km/h)' 'Speed limit (50km/h)'
 'Speed limit (60km/h)' 'Speed limit (70km/h)' 'Speed limit (80km/h)'
 'End of speed limit (80km/h)' 'Speed limit (100km/h)'
 'Speed limit (120km/h)' 'No passing'
 'No passing for vehicles over 3.5 metric tons'
 'Right-of-way at the next intersection' 'Priority road' 'Yield' 'Stop'
 'No vehicles' 'Vehicles over 3.5 metric tons prohibited' 'No entry'
 'General caution' 'Dangerous curve to the left'
 'Dangerous curve to the right' 'Double curve' 'Bumpy road'
 'Slippery road' 'Road narrows on the right' 'Road work' 'Traffic signals'
 'Pedestrians' 'Children crossing' 'Bicycles crossing'
 'Beware of ice/snow' 'Wild animals crossing'
 'End of all speed and passing limits' 'Turn right ahead'
 'Turn left ahead' 'Ahead only' 'Go straight or right'
 'Go straight or left' 'Keep right' 'Keep left' 'Roundabout mandatory'
 'End of no passing' 'End of no passing by vehicles over 3.5 metric']
{0: 0, 1: 0, 2: 0, 3: 0

In [109]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout

# Assuming image dimensions
height = 32  # example height
width = 32   # example width
channels = 3  # RGB channels

# Input layer
inputs = Input(shape=(height, width, channels))

# Convolutional layers
x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Shared dense layer
shared_dense = Dense(128, activation='relu')(x)
shared_dense = Dropout(0.5)(shared_dense)

# Head 1: Binary classification (street sign or no street sign)
head_1 = Dense(64, activation='relu')(shared_dense)
head_1 = Dropout(0.5)(head_1)
binary_output = Dense(1, activation='sigmoid', name='binary_output')(head_1)

# Head 2: Multi-class classification (4 different shapes)
head_2 = Dense(64, activation='relu')(shared_dense)
head_2 = Dropout(0.5)(head_2)
shape_output = Dense(4, activation='softmax', name='shape_output')(head_2)

# Create the model
model = Model(inputs=inputs, outputs=[binary_output, shape_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'binary_output': 'binary_crossentropy', 
                    'shape_output': 'sparse_categorical_crossentropy'}, 
              metrics=['accuracy']
              )

# Model summary
model.summary()

print("Finished building model...")

Model: "functional_51"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, 32, 32, 3) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_75 (Conv2D)  │ (None, 32, 32,    │     896 │ input_layer_25[0][0] │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_75    │ (None, 16, 16,    │       0 │ conv2d_75[0][0]      │
│ (MaxPooling2D)      │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 16, 16,    │  18,496 │ max_pooling2d_75[0]… │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_76    │ (None, 8, 8, 64)  │       0 │ conv2d_76[0][0]      │
│ (MaxPooling2D)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 8, 8, 128) │  73,856 │ max_pooling2d_76[0]… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_77    │ (None, 4, 4, 128) │       0 │ conv2d_77[0][0]      │
│ (MaxPooling2D)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_25          │ (None, 2048)      │       0 │ max_pooling2d_77[0]… │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_70 (Dense)    │ (None, 128)       │ 262,272 │ flatten_25[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_52          │ (None, 128)       │       0 │ dense_70[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_71 (Dense)    │ (None, 64)        │   8,256 │ dropout_52[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_72 (Dense)    │ (None, 64)        │   8,256 │ dropout_52[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_53          │ (None, 64)        │       0 │ dense_71[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_54          │ (None, 64)        │       0 │ dense_72[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ binary_output       │ (None, 1)         │      65 │ dropout_53[0][0]     │
│ (Dense)             │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ shape_output        │ (None, 4)         │     260 │ dropout_54[0][0]     │
│ (Dense)             │                   │         │                      │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 372,357 (1.42 MB)

 Trainable params: 372,357 (1.42 MB)

 Non-trainable params: 0 (0.00 B)

Finished building model...


In [110]:
validation_file = './Data/valid.p'

with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
images_valid, labels_valid = valid['features'], valid['labels']

forms_valid = []
for i in range(len(labels_valid)):
    forms_valid.append(sign_dict.get(labels_valid[i]))
    if labels_valid[i] < 9:
        labels_valid[i] = 0
    elif labels_valid[i] >= 9:
        labels_valid[i] = 1

In [111]:
import numpy

forms_train = numpy.array(forms_train)
forms_valid = numpy.array(forms_valid)

print("Img train: ", len(images_train), images_train.dtype)
print("Form train: ", len(forms_train), type(forms_train))
print("Label train: ", len(labels_train), type(labels_train))

print("Img valid: ", len(images_valid), images_valid.dtype)
print("Form valid: ", len(forms_valid), type(forms_valid))
print("Label valid: ", len(labels_valid), type(labels_valid))

train_data = {'form_output': forms_train, 'type_output': labels_train}
validation_data = {'form_output': forms_valid, 'type_output': labels_valid}

# Train the model
history = model.fit(images_train, 
                    train_data, 
                    epochs=10, 
                    batch_size=32, 
                    validation_data=(images_valid, validation_data))

Img train:  34799 uint8
Form train:  34799 <class 'numpy.ndarray'>
Label train:  34799 <class 'numpy.ndarray'>
Img valid:  4410 uint8
Form valid:  4410 <class 'numpy.ndarray'>
Label valid:  4410 <class 'numpy.ndarray'>
Epoch 1/10


ValueError: No loss to compute. Provide a `loss` argument in `compile()`.

In [ ]:
# Define the file name for saving the model
model_filename = 'Multi-head-model'

# Save the model to a file
model.save(model_filename)